In [1]:
import os
import pandas as pd
import json
from datetime import datetime

In [2]:
probationary_period = 150
thresholds = json.load(open(f"./thresholds.json"))

In [3]:

# results_df = pd.DataFrame(columns=["Datafile", "fault_injection_timestamp", "vmft-lad_detection_idx", "vmft-lad_detection_time", 
#                                    "numenta_detection_idx", "numenta_detection_time", "failure_timestamp", 
#                                    "vmft-lad_time_before_failure", "numenta_time_before_failure"
#                                    ]).set_index("Datafile", drop=True)

# results = {}

# for model_name in ["vmft-lad", "numenta"]:

#     threshold = thresholds[model_name]["standard"]["threshold"]

#     dataset_name = "hdd"

#     dataset_dir = f"../data/{dataset_name}"
#     dataset_metadata = json.load(
#         open(f"{dataset_dir}/dataset_metadata.json"))

#     for resultFile in os.scandir(f"./{model_name}/{dataset_name}"):
#         if not resultFile.name.endswith(".csv"):
#             continue

#         dataset_result_df = pd.read_csv(resultFile.path, header=0)
#         data_file_name = resultFile.name[len(model_name) + 1: -4]

#         label_index = dataset_metadata[data_file_name]["label_region"] - 1
#         failure_point_index = dataset_metadata[data_file_name]["failure_point"]

#         fault_injection_timestamp = dataset_metadata[data_file_name]["label_region_timestamp"]
#         failure_timestamp = dataset_metadata[data_file_name]["failure_point_timestamp"]

#         try:
#             results[data_file_name]
#         except KeyError:
#             results[data_file_name] = {}
            
#         results[data_file_name]["fault_injection_timestamp"] = fault_injection_timestamp
#         results[data_file_name]["failure_timestamp"] = failure_timestamp

#         fault_injection_time = None
#         if not fault_injection_timestamp == "":
#             fault_injection_time = pd.to_datetime(
#                 fault_injection_timestamp)

#         failure_time = None
#         if not failure_timestamp == "":
#             failure_time = pd.to_datetime(failure_timestamp)


#         i = label_index  # iterator for the data points
#         # # Benign region
#         # while (i < label_index):
#         #     anomaly_score = dataset_result_df.iloc[i]["anomaly_score"]
#         #     if anomaly_score >= threshold:
#         #         false_positive += 1
#         #     else:
#         #         true_negative += 1
#         #     i += 1

#         # Anomalous region
#         while (i < failure_point_index):
#             anomaly_score = dataset_result_df.iloc[i]["anomaly_score"]
#             if anomaly_score >= threshold:  # Early detection
#                 # early_detection_count += 1

#                 results[data_file_name][f"{model_name}_detection_idx"] = i

#                 detection_timestamp = dataset_result_df.iloc[i]["timestamp"]
#                 detection_time = pd.to_datetime(detection_timestamp)

#                 results[data_file_name][f"{model_name}_detection_time"] = detection_time

#                 # time_to_detection = (
#                 #     detection_time - fault_injection_time).milliseconds

#                 time_before_failure = (failure_time - detection_time).seconds
                
#                 results[data_file_name][f"{model_name}_time_before_failure"] = round(time_before_failure/60, 3)

#                 break
#             i += 1

In [4]:
# results_df = pd.DataFrame(results).T

In [5]:
# results_df.to_csv("./results.csv")

In [6]:
def execute(model_name: str, llmOnly: bool = False):

    threshold = thresholds[model_name]["standard"]["threshold"]
    print(f"Model: {model_name}, Threshold: {threshold}")
    # Average False positive rate
    # Average time for first detection after fault injection
    false_positive = 0
    true_negative = 0

    early_detection_count = 0
    faulty_dataset_count = 0

    average_time_to_detection = {}
    average_detection_time_before_failure = {}

    datasets = ["benign", "hdd", "oom", "buffer-io", "cpu"]

    for dataset_name in datasets:
        dataset_dir = f"../data/{dataset_name}"
        dataset_metadata = json.load(open(f"{dataset_dir}/dataset_metadata.json"))

        average_time_to_detection[dataset_name] = None
        average_detection_time_before_failure[dataset_name] = None

        datasetResultDir = (
            f"./LLM_only/{model_name}/{dataset_name}"
            if llmOnly
            else f"./{model_name}/{dataset_name}"
        )
        for resultFile in os.scandir(datasetResultDir):
            if not resultFile.name.endswith(".csv"):
                continue

            dataset_result_df = pd.read_csv(resultFile.path, header=0)
            data_file_name = resultFile.name[len(model_name) + 1 : -4]

            label_index = dataset_metadata[data_file_name]["label_region"] - 1
            failure_point_index = dataset_metadata[data_file_name]["failure_point"]

            fault_injection_timestamp = dataset_metadata[data_file_name][
                "label_region_timestamp"
            ]
            failure_timestamp = dataset_metadata[data_file_name][
                "failure_point_timestamp"
            ]

            fault_injection_time = None
            if not fault_injection_timestamp == "":
                fault_injection_time = pd.to_datetime(fault_injection_timestamp)

            failure_time = None
            if not failure_timestamp == "":
                failure_time = pd.to_datetime(failure_timestamp)

            # for benign datasets, the label_index is 0
            if label_index <= 0:
                label_index = len(dataset_result_df)
                failure_point_index = len(dataset_result_df)
            else:
                faulty_dataset_count += 1

            i = probationary_period + 1  # iterator for the data points
            # Benign region
            while i < label_index:
                anomaly_score = dataset_result_df.iloc[i]["anomaly_score"]
                if anomaly_score >= threshold:
                    false_positive += 1
                else:
                    true_negative += 1
                i += 1

            # Anomalous region
            while i < failure_point_index:
                anomaly_score = dataset_result_df.iloc[i]["anomaly_score"]
                if anomaly_score >= threshold:  # Early detection
                    early_detection_count += 1

                    detection_timestamp = dataset_result_df.iloc[i]["timestamp"]
                    detection_time = pd.to_datetime(detection_timestamp)

                    # print( "Dataset: ", data_file_name, "  Detection at: ", i, "  Time: ", detection_time)

                    time_to_detection = (
                        detection_time - fault_injection_time
                    ).microseconds / 1000
                    time_before_failure = (failure_time - detection_time).seconds / 60

                    # print("Time before failure: ", time_before_failure/60, " minutes")

                    average_time_to_detection[dataset_name] = (
                        time_to_detection
                        if average_time_to_detection[dataset_name] is None
                        else (
                            average_time_to_detection[dataset_name] + time_to_detection
                        )
                        / 2
                    )
                    average_detection_time_before_failure[dataset_name] = (
                        time_before_failure
                        if average_detection_time_before_failure[dataset_name] is None
                        else (
                            average_detection_time_before_failure[dataset_name]
                            + time_before_failure
                        )
                        / 2
                    )
                    break
                i += 1

    false_positive_rate = false_positive / (false_positive + true_negative)
    print(f"False positive count: {false_positive}")
    print(f"False positive rate: {round(false_positive_rate, 4)}")
    print(
        f"Early detection rate: {round(early_detection_count/faulty_dataset_count, 4)}"
    )
    print(f"Average time to detection - milliseconds: {average_time_to_detection}")
    print(
        f"Average detection time before failure - minutes: {average_detection_time_before_failure}"
    )

In [6]:
execute("vmft-lad")

Model: vmft-lad, Threshold: 1.0
False positive count: 113
False positive rate: 0.0002
Early detection rate: 0.9628
Average time to detection - milliseconds: {'benign': None, 'hdd': 968.7480775766085, 'oom': 468.7344080342906, 'buffer-io': 968.199120681212, 'cpu': 8.75600554431943}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.651849923786019, 'oom': 3.0338102298355163, 'buffer-io': 13.33859723954551, 'cpu': 14.535446894853292}


In [8]:
execute("numenta")

Model: numenta, Threshold: 0.504963404813
False positive count: 414
False positive rate: 0.0007
Early detection rate: 0.6208
Average time to detection - milliseconds: {'benign': None, 'hdd': 517.5647055581534, 'oom': 904.6745581054687, 'buffer-io': 0.24440863060355014, 'cpu': 1.0}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.675002919676338, 'oom': 1.5358723958333333, 'buffer-io': 13.354729834212039, 'cpu': 8.46253013934012}


In [9]:
execute("knncad")

Model: knncad, Threshold: 1.0
False positive count: 4570
False positive rate: 0.0074
Early detection rate: 0.7658
Average time to detection - milliseconds: {'benign': None, 'hdd': 921.8663295289207, 'oom': 614.3773518218993, 'buffer-io': 999.9379649047843, 'cpu': 1.0}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.901461066853159, 'oom': 2.0129403750101726, 'buffer-io': 12.073575605253577, 'cpu': 12.555667555433445}


In [10]:
execute("ARTime")

Model: ARTime, Threshold: 0.308129
False positive count: 1485
False positive rate: 0.0024
Early detection rate: 0.7825
Average time to detection - milliseconds: {'benign': None, 'hdd': 517.5690119895522, 'oom': 238.27074967507554, 'buffer-io': 997.9805484542189, 'cpu': 2.26953174360115}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.674999271620011, 'oom': 4.057058896744176, 'buffer-io': 13.337871817296088, 'cpu': 13.120237247275885}


In [11]:
execute("expose")

Model: expose, Threshold: 0.9720854386352024
False positive count: 2377
False positive rate: 0.0039
Early detection rate: 0.8513
Average time to detection - milliseconds: {'benign': None, 'hdd': 483.4261965567764, 'oom': 182.70244430208794, 'buffer-io': 999.999, 'cpu': 9.0}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.658399279454928, 'oom': 3.1366656689038077, 'buffer-io': 11.864066392377282, 'cpu': 23.95914125818031}


In [7]:
execute("DeepLog")

Model: DeepLog, Threshold: 1.0
False positive count: 2292
False positive rate: 0.0037
Early detection rate: 1.0
Average time to detection - milliseconds: {'benign': None, 'hdd': 500.50304005984765, 'oom': 8.515709403009705, 'buffer-io': 0.48828076172585544, 'cpu': 9.0}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.675287281052338, 'oom': 4.376496198794549, 'buffer-io': 13.354725769672456, 'cpu': 23.95914125818031}


In [21]:
execute("LLM-BartLarge", llmOnly=True)

Model: LLM-BartLarge, Threshold: 1.0
False positive count: 107515
False positive rate: 0.1749
Early detection rate: 1.0
Average time to detection - milliseconds: {'benign': None, 'hdd': 960.9317632159788, 'oom': 8.257854224667707, 'buffer-io': 7.105421310963972e-12, 'cpu': 8.999999984633176}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.651980195837822, 'oom': 4.376496198794549, 'buffer-io': 13.354733907693289, 'cpu': 23.959141258189042}


In [22]:
execute("LLM-GPT3", llmOnly=True)

Model: LLM-GPT3, Threshold: 1.0
False positive count: 327
False positive rate: 0.0005
Early detection rate: 1.0
Average time to detection - milliseconds: {'benign': None, 'hdd': 960.9317632159805, 'oom': 8.257854701504865, 'buffer-io': 7.105421310963972e-12, 'cpu': 9.0}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.651980195837822, 'oom': 4.376496198794549, 'buffer-io': 13.354733907693289, 'cpu': 23.95914125818031}


In [7]:
execute("LLM-Cyrax7B", llmOnly=True)

Model: LLM-Cyrax7B, Threshold: 1.0
False positive count: 53983
False positive rate: 0.0878
Early detection rate: 1.0
Average time to detection - milliseconds: {'benign': None, 'hdd': 3.9291493913899584, 'oom': 3.3892366200352937, 'buffer-io': 7.1054204639318715e-12, 'cpu': 8.998046875}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.68356352083972, 'oom': 4.376496198794549, 'buffer-io': 13.354733907693289, 'cpu': 23.959173810263643}


In [8]:
execute("LLM-EmertonMonarch7B", llmOnly=True)

Model: LLM-EmertonMonarch7B, Threshold: 1.0
False positive count: 74216
False positive rate: 0.1207
Early detection rate: 1.0
Average time to detection - milliseconds: {'benign': None, 'hdd': 3.9291493913899584, 'oom': 8.10044179246803, 'buffer-io': 7.1054204639318715e-12, 'cpu': 8.998046875}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.68356352083972, 'oom': 4.424022240461215, 'buffer-io': 13.354733907693289, 'cpu': 23.959173810263643}


In [9]:
execute("LLM-Falcon7B", llmOnly=True)

Model: LLM-Falcon7B, Threshold: 1.0
False positive count: 48859
False positive rate: 0.0795
Early detection rate: 0.9907
Average time to detection - milliseconds: {'benign': None, 'hdd': 960.9322400526192, 'oom': 5.846148464394352, 'buffer-io': 999.999, 'cpu': 8.998046875}
Average detection time before failure - minutes: {'benign': None, 'hdd': 15.665530120045299, 'oom': 4.418162865461215, 'buffer-io': 12.25917505414257, 'cpu': 23.959173810263643}
